In [1]:
import sys
import torch
print(torch.cuda.device_count())
sys.path.append("..")
import numpy as np
import os
import pickle, importlib, random, Engine, tqdm, copy, json, time, argparse
import util.Generator as Generator
import util.Datahelper as dh

sys.argv = ' '
parser = argparse.ArgumentParser()
parser.add_argument('-test', dest='test', default=False, required=False)
parser.add_argument('-ckptname', dest='ckptname', default=None, required=False)
parser.add_argument('-cityname', dest='cityname', default='TKY', required=False)

parser.add_argument('-nl', dest='num_layers', default=2, required=False)
parser.add_argument('-nhd', dest='num_hidden_dims', default=2**8, required=False)
parser.add_argument('-nh', dest='num_heads', default=4, required=False)

parser.add_argument('-i', dest='use_item_feat', default=True, required=False)
parser.add_argument('-u', dest='use_user_feat', default=True, required=False)

parser.add_argument(
    '-pt_sample_func', dest='pt_sample_func', default='(lambda x:x)', required=False)
parser.add_argument(
    '-pt_sample_param', dest='pt_sample_param', default='0', required=False)
parser.add_argument(
    '-pt_history_func', dest='pt_history_func', default='(lambda x:x)', required=False)
parser.add_argument(
    '-pt_history_param', dest='pt_history_param', default='100', required=False)

parser.add_argument('-batch_size', dest='batch_size', default=16, required=False)
parser.add_argument('-nonimprove_limit', dest='nonimprove_limit', default=10, required=False)
parser.add_argument('-seed', dest='seed', default=0, required=False, type=int)
args = parser.parse_args()
print(args)

Engine.set_random_seed(args.seed)

feat = {'NYC':[9989, 251, 45, 60], 'TKY':[7873, 247, 37, 45]}[args.cityname]

basic_config = {
    'name':'checkin-{}'.format(args.cityname),
    'cuda_num' : Engine.GPU_max_free_memory(),
    'course_file' : '../datasets/Checkin_data/{}_dataset.pkl'.format(args.cityname),
    'num_times' : 100,
    'num_items' : feat[0], 
    'batch_size' : args.batch_size, 
    'feats' : feat[1:]
}
    
save_name = 'checkpoint/KNN'
print(basic_config)

os.environ['CUDA_VISIBLE_DEVICES'] = str(basic_config['cuda_num'])
with open(basic_config['course_file'], 'rb') as f:
    user_dict = pickle.load(f)
    print('Total Number of Users : ' + str(len(user_dict)))
    
all_keys = list(user_dict.keys())
all_keys.sort()
np.random.shuffle(all_keys)

train_keys, tv_keys = dh.list_partition(all_keys, 0.7, seed=0)
test_keys, valid_keys = dh.list_partition(tv_keys, 0.5, seed=0)

8
Namespace(batch_size=16, cityname='TKY', ckptname=None, nonimprove_limit=10, num_heads=4, num_hidden_dims=256, num_layers=2, pt_history_func='(lambda x:x)', pt_history_param='100', pt_sample_func='(lambda x:x)', pt_sample_param='0', seed=0, test=False, use_item_feat=True, use_user_feat=True)
{'name': 'checkin-TKY', 'cuda_num': 2, 'course_file': '../datasets/Checkin_data/TKY_dataset.pkl', 'num_times': 100, 'num_items': 7873, 'batch_size': 16, 'feats': [247, 37, 45]}
Total Number of Users : 2192


In [2]:
def generator2feature(generator):
    dataset = generator.__getitem__(batch_id=0, batch_size='MAX')[0]
    course = dataset[0]
    target = dataset[-1]
    return course, target

if True:
    train_generator_config = {
        'name' : None,
        'training' : True, 
        'sample_func' : args.pt_sample_func,
        'sample_param' : args.pt_sample_param,
        'history_func' : args.pt_history_func,
        'history_param' : args.pt_history_param,
        'next_basket' : True, 
        'batch_size' : basic_config['batch_size'],
        'shuffle' : True,
        'fixed_seed' : False}

    train_generator = Generator.TimeMultihotGenerator(
        user_dict, train_keys, basic_config, train_generator_config)

train_courses, train_target = generator2feature(train_generator)

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import NearestNeighbors
import util.Metrics as Metrics

def userKNN_GPU(train_features, test_courses, cuda_num):
    # train_features, torch.FloatTensor in GPU [num_stu, num_courses]
    # test_courses, np.array in CPU [num_stu, num_sem, num_courses]
    test_features = torch.FloatTensor(test_courses.sum(1).astype(float)).cuda(cuda_num)
    #train_features += torch.rand(train_features.shape).cuda(cuda_num) * 1e-10
    #test_features += torch.rand(test_features.shape).cuda(cuda_num) * 1e-10

    sim = []
    for iter in tqdm.tqdm(range(test_features.shape[0])):
        sim.append(torch.cosine_similarity(train_features.unsqueeze(1), test_features[iter].unsqueeze(0).unsqueeze(0), dim=-1))
    sim = torch.cat(sim, dim=-1).T

    pred = []
    for iter in range(basic_config['num_times']):
        pred.append(torch.matmul(sim, torch.FloatTensor(train_courses[:, iter]).cuda(cuda_num))[:, np.newaxis])
    pred = torch.cat(pred, dim=1)
    #pred += torch.rand(pred.shape).cuda(cuda_num) * 1e-10
    pred = pred.cpu().numpy()
    return pred

In [3]:
test_generator_config = {
    'training' : False, 
    'max_sampling' : 50,
    'mask_rate' : None,
    'historical' : None,
    'batch_size' : 16,
    'shuffle' : False,
    'fixed_seed' : True}

results_mat = {}
for h in [0,1,2,3,5,7,10] + list(range(20, 81, 20)):
    results_mat[h] = {}
    for r in [0,1,2,3,5,7] + list(range(10, 51, 10)):
        test_generator_config['sample_func'] = '(lambda x:x)'
        test_generator_config['sample_param'] = str(r)
        test_generator_config['history_func'] = '(lambda x:x)'
        test_generator_config['history_param'] = str(h)
        test_generator_config['name'] = 'H={1}_R={0}'.format(r, h)
        test_generator = Generator.TimeMultihotGenerator(
            user_dict, test_keys, basic_config, test_generator_config)
        print(test_generator.name + ' ' + str(test_generator.batch_size))
        train_features = torch.FloatTensor(train_courses.sum(1).astype(float))
        train_features_cuda = train_features.cuda(basic_config['cuda_num'])
        test_courses, test_target = generator2feature(test_generator)
        user_pred = userKNN_GPU(train_features_cuda, test_courses, basic_config['cuda_num'])
        recall, recall_per_sem = Metrics.recall(test_target[:, h:], user_pred[:, h:], at_n=10)
        
        print('Recall: {:.4f}'.format(recall))
        print('Recall per semester: {}'.format(recall_per_sem.round(4)))
        results_mat[h][r] = [recall, recall_per_sem]
save_name = save_name + '.npy'
print(save_name)
np.save(save_name, np.array(results_mat))

H=0_R=0 16


100%|██████████| 329/329 [00:00<00:00, 2151.99it/s]


Recall: 0.0015
Recall per semester: [0.002  0.     0.     0.0025 0.0021 0.002  0.     0.0018 0.0018 0.0018
 0.     0.0034 0.0019 0.0018 0.0018 0.     0.0038 0.0019 0.0038 0.
 0.002  0.     0.0021 0.0019 0.0039 0.0019 0.0037 0.004  0.0038 0.
 0.004  0.     0.002  0.002  0.     0.0021 0.0039 0.     0.0044 0.
 0.     0.     0.     0.0024 0.0023 0.0068 0.     0.     0.     0.0045
 0.     0.     0.     0.     0.0059 0.     0.     0.     0.0029 0.
 0.0029 0.0031 0.003  0.     0.     0.     0.     0.     0.0033 0.
 0.     0.0034 0.     0.     0.     0.     0.     0.     0.     0.0041
 0.     0.0043 0.0071 0.     0.     0.0048 0.     0.     0.     0.
 0.     0.     0.     0.     0.0052 0.     0.     0.     0.     0.    ]
H=0_R=1 16


100%|██████████| 329/329 [00:00<00:00, 2109.43it/s]


Recall: 0.1075
Recall per semester: [0.127  0.1258 0.1207 0.1007 0.1148 0.102  0.1079 0.1191 0.1184 0.1083
 0.1065 0.1092 0.0969 0.1085 0.1119 0.1259 0.1176 0.1456 0.113  0.1255
 0.1098 0.1226 0.1104 0.1269 0.107  0.1224 0.098  0.1255 0.1283 0.1144
 0.11   0.1473 0.0986 0.0925 0.1139 0.1141 0.1066 0.1263 0.1173 0.0859
 0.1033 0.1204 0.1119 0.1351 0.093  0.1159 0.0911 0.0946 0.1198 0.1047
 0.1092 0.1181 0.1153 0.133  0.1246 0.0994 0.0926 0.1015 0.1121 0.0775
 0.0826 0.0734 0.1012 0.1019 0.1101 0.1095 0.1006 0.1366 0.0792 0.1046
 0.1062 0.0774 0.0909 0.0816 0.0867 0.0876 0.0839 0.0923 0.0758 0.0779
 0.1066 0.087  0.0783 0.1274 0.0952 0.0673 0.0889 0.0872 0.0714 0.0848
 0.0805 0.0856 0.1158 0.0952 0.0942 0.1034 0.0751 0.0737 0.0941 0.0964]
H=0_R=2 16


...

Recall: 0.1968
Recall per semester: [0.2316 0.2087 0.2028 0.2278 0.2035 0.1779 0.1667 0.2018 0.183  0.1741
 0.2184 0.2032 0.1895 0.1786 0.199  0.1609 0.2254 0.1895 0.1765 0.1827]
H=80_R=40 16


100%|██████████| 329/329 [00:00<00:00, 2167.26it/s]


Recall: 0.1966
Recall per semester: [0.2279 0.2087 0.2028 0.2278 0.2035 0.1779 0.1667 0.2018 0.183  0.1741
 0.2184 0.2032 0.1895 0.1786 0.199  0.1609 0.2254 0.1895 0.1765 0.1827]
H=80_R=50 16


100%|██████████| 329/329 [00:00<00:00, 2169.74it/s]


Recall: 0.1973
Recall per semester: [0.2316 0.2087 0.2064 0.2278 0.2035 0.1731 0.1722 0.1972 0.183  0.1741
 0.2184 0.2032 0.1895 0.1786 0.2042 0.1667 0.2254 0.1895 0.1765 0.1827]
checkpoint/KNN.npy
